# Análise Avançada de Séries Temporais Climáticas com Foco em Erosão

Este notebook realiza análises estatísticas avançadas na série de precipitação de 20 anos (2005-2025) com foco em:
- Identificação de eventos extremos de precipitação
- Cálculo de índices de erosividade (EI30)
- Modelagem SARIMA para previsão sazonal
- Testes de homogeneidade para mudanças climáticas

**Base de Dados:** `serie_precipitacao_20anos.csv`

In [3]:
# Configuração inicial
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pymannkendall import original_test
import warnings

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.dpi'] = 300

# Diretórios
BASE_DIR = Path.cwd().parent
FIGURAS_DIR = BASE_DIR / "figuras"
DADOS_DIR = BASE_DIR / "dados"

# Carregar dados
df = pd.read_csv(DADOS_DIR / "serie_precipitacao_20anos.csv", parse_dates=['Data'], index_col='Data')
df = df.rename(columns={'Precipitacao_mm': 'precipitacao'})
print(f"Dados carregados: {df.shape[0]} registros de {df.index.min().date()} a {df.index.max().date()}")

Dados carregados: 7335 registros de 2005-11-01 a 2025-11-30


## 1. Identificação de Eventos Extremos de Precipitação

Eventos extremos de precipitação são críticos para estudos de erosão, pois causam a maior parte da perda de solo. Vamos identificar:
- Eventos acima do percentil 95%
- Duração e intensidade de cada evento
- Precipitação acumulada por evento

In [4]:
# Identificar eventos extremos (percentil 95%)
limiar = np.percentile(df['precipitacao'], 95)
eventos = df[df['precipitacao'] > limiar].reset_index()

if not eventos.empty:
    # Calcular diferença entre dias consecutivos
    eventos['diff'] = eventos['Data'].diff().dt.days.fillna(0)
    
    # Identificar grupos de eventos consecutivos
    eventos['grupo'] = (eventos['diff'] > 1).cumsum()
    
    # Calcular características de cada evento
    resumo_eventos = eventos.groupby('grupo').agg(
        inicio=('Data', 'min'),
        fim=('Data', 'max'),

        duracao=('Data', lambda x: (x.max() - x.min()).days + 1),    print("Nenhum evento extremo encontrado")

        precipitacao_total=('precipitacao', 'sum'),else:

        intensidade_max=('precipitacao', 'max')    display(resumo_eventos.head(10))

    ).reset_index(drop=True)    print(f"Limiar de precipitação: {limiar:.2f} mm")

        print(f"Identificados {len(resumo_eventos)} eventos extremos")

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1126420485.py, line 17)

## 2. Índices de Erosividade (EI30)

O índice EI30 é um indicador do potencial erosivo da chuva, calculado como:

$$EI30 = \sum (E \cdot I_{30})$$

Onde:
- $E$ = energia cinética da chuva (MJ/ha/mm)
- $I_{30}$ = intensidade máxima em 30 minutos (mm/h)

In [ ]:
# Cálculo do EI30 mensal
def calcular_ei30(serie):
    """Calcula o índice de erosividade mensal"""
    # Calcular energia cinética (MJ/ha/mm)
    ec = 0.29 * (1 - 0.72 * np.exp(-0.05 * serie))
    
    # Calcular I30 (intensidade máxima em 30 minutos)
    i30 = serie.resample('D').max().resample('M').max()
    
    # EI30 = soma(ec * precipitacao) * i30
    ei30 = (ec * serie).resample('M').sum() * i30
    return ei30

ei30 = calcular_ei30(df['precipitacao'])

# Plotar série temporal do EI30
plt.figure(figsize=(12, 6))
ei30.plot(title='Índice de Erosividade (EI30) Mensal')
plt.ylabel('EI30 (MJ·mm/ha·h)')
plt.xlabel('Data')
plt.grid(True)
plt.tight_layout()
plt.savefig(FIGURAS_DIR / "indice_erosividade_mensal.png")
plt.show()

## 3. Análise de Tendências Climáticas

Aplicamos:
- Teste de Mann-Kendall para identificar tendências
- Estimador de Sen para quantificar a magnitude das tendências
- Decomposição sazonal para separar componentes

In [ ]:
# Análise de tendências com Mann-Kendall
mensal = df['precipitacao'].resample('M').sum()

# Teste de Mann-Kendall
mk_result = original_test(mensal.values)
print(f"Tendência: {mk_result.trend}")
print(f"p-valor: {mk_result.p:.4f}")

# Sen's Slope
slopes = []
n = len(mensal)
for i in range(n):
    for j in range(i+1, n):
        slopes.append((mensal[j] - mensal[i]) / (j - i))
sen_slope = np.median(slopes)
print(f"Sen's Slope: {sen_slope:.4f} mm/mês/ano")

# Decomposição sazonal
dec = sm.tsa.seasonal_decompose(mensal, model='additive', period=12)

# Plotar decomposição
fig = dec.plot()
fig.set_size_inches(12, 8)
fig.suptitle('Decomposição Sazonal da Precipitação Mensal')
plt.tight_layout()
plt.savefig(FIGURAS_DIR / "decomposicao_sazonal.png")
plt.show()

## 4. Modelagem SARIMA para Previsão Sazonal

Modelo SARIMA (Seasonal ARIMA) para previsão de precipitação com componentes sazonais:

$$SARIMA(p,d,q)(P,D,Q)_s$$

Onde:
- $p$: ordem AR
- $d$: diferenciação
- $q$: ordem MA
- $P,D,Q,s$: componentes sazonais

In [ ]:
# Modelagem SARIMA
modelo = SARIMAX(mensal, order=(1,0,1), seasonal_order=(1,1,1,12))
resultado = modelo.fit(disp=False)

# Previsão para 2 anos
previsao = resultado.get_forecast(steps=24)
previsao_media = previsao.predicted_mean
intervalo_conf = previsao.conf_int()

# Plotar previsão
plt.figure(figsize=(12, 6))
plt.plot(mensal.index, mensal, label='Observado')
plt.plot(previsao_media.index, previsao_media, color='r', label='Previsão')
plt.fill_between(intervalo_conf.index, 
                 intervalo_conf.iloc[:,0], 
                 intervalo_conf.iloc[:,1], 
                 color='pink', alpha=0.3)
plt.title('Previsão de Precipitação Mensal (SARIMA)')
plt.ylabel('Precipitação (mm)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(FIGURAS_DIR / "previsao_sarima.png")
plt.show()

## 5. Testes de Homogeneidade

Verificamos se há mudanças significativas na série temporal usando:
- Teste de Pettitt para ponto de mudança
- Teste de Buishand para quebras de homogeneidade

In [ ]:
# Teste de Pettitt para ponto de mudança
from scipy import stats

def teste_pettitt(serie):
    n = len(serie)
    U = np.zeros(n)
    
    for t in range(n):
        U[t] = np.sum(np.sign(serie[t] - serie[:t]))
    
    K = np.max(np.abs(U))
    ponto_mudanca = np.argmax(np.abs(U))
    p = 2 * np.exp(-6 * K**2 / (n**3 + n**2))
    
    return ponto_mudanca, p

ponto_mudanca, p_valor = teste_pettitt(mensal)
data_mudanca = mensal.index[ponto_mudanca]

print(f"Ponto de mudança detectado em: {data_mudanca.strftime('%Y-%m')}")
print(f"p-valor: {p_valor:.4f}")

# Plotar série com ponto de mudança
plt.figure(figsize=(12, 6))
plt.plot(mensal.index, mensal, label='Precipitação')
plt.axvline(x=data_mudanca, color='r', linestyle='--', 
            label=f'Ponto de Mudança (p={p_valor:.4f})')
plt.title('Série Temporal com Ponto de Mudança Detectado')
plt.ylabel('Precipitação (mm)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(FIGURAS_DIR / "ponto_mudanca.png")
plt.show()

## Conclusões e Recomendações

Principais achados:
1. Identificação de X eventos extremos com potencial erosivo
2. Tendência [significativa/não significativa] na precipitação
3. Ponto de mudança climática detectado em [ano]
4. Previsão sazonal para os próximos 2 anos

**Recomendações para estudos de erosão:**
- Focar em eventos acima de Y mm
- Monitorar especialmente o período [meses] de maior erosividade
- Considerar mudanças no regime de chuvas a partir de [ano]